# 全量微调GPT

### 1. 下载数据集

In [1]:
from datasets import load_dataset

# 指定数据集缓存目录
cache_dir = "/raid/gfc/llm/datasets/ChinesePoems"

# 加载数据集
ds = load_dataset("larryvrh/Chinese-Poems", cache_dir=cache_dir)

# 查看数据集的基本信息
print("数据集结构：")
print(ds)

# 获取数据集的大小
print(f"\n训练集大小: {len(ds['train'])}")

# 查看数据集的列名（特征）
print("\n数据集的特征：")
print(ds['train'].features)


/raid/gfc/anaconda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


数据集结构：
DatasetDict({
    train: Dataset({
        features: ['dynasty', 'author', 'title', 'content'],
        num_rows: 217561
    })
})

训练集大小: 217561

数据集的特征：
{'dynasty': Value(dtype='string', id=None), 'author': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}


In [2]:
import os

# 如果数据集未处理，对数据集进行处理
save_path = "/raid/gfc/llm/datasets/ChinesePoems/poems.txt"
if not os.path.exists(save_path):
    poems = []

    for poem in ds['train']:
        content = poem['content']
        poems.append(content.replace('\n', ''))


    # 将诗句写入文件
    with open(save_path, 'w', encoding='utf-8') as f:
        for poem in poems:
            f.write(poem + '\n')


### 2. 定义MyDataset

In [3]:
from torch.utils.data import Dataset

# 制作 Dataset
class MyDataset(Dataset):
    def __init__(self, data_path):
        with open(data_path, 'r', encoding='utf-8') as f:
            self.data = f.readlines()
        self.data = [line.strip() for line in self.data]
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [4]:
# 加载数据集
dataset = MyDataset(save_path)

print(len(dataset), dataset[0])

217561 青鞋踏尽剑鋩山，借枕僧房落照间。高屋凭虚听泉语，岭云应似我身闲。


### 3. 加载模型

In [5]:
import torch
import subprocess

# GPU选择函数
def pick_free_gpu(start=7, end=0, memory_threshold=100):
    """
    自动选择空闲的GPU
    :param start: 起始GPU编号
    :param end: 结束GPU编号
    :param memory_threshold: 显存占用阈值（MB），低于此值认为GPU空闲
    :return: torch.device对象
    """
    try:
        # 获取nvidia-smi输出，包含显存使用和GPU利用率
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used,utilization.gpu', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        
        # 解析输出
        gpu_info = []
        for line in result.strip().split('\n'):
            memory, util = map(int, line.split(', '))
            gpu_info.append((memory, util))
        
        print("当前GPU状态：")
        for i, (memory, util) in enumerate(gpu_info):
            print(f"GPU {i}: 显存使用 {memory}MB, 利用率 {util}%")
        
        # 从start到end检查GPU（包括end）
        for i in range(start, end-1, -1):
            if 0 <= i < len(gpu_info):  # 确保i在有效范围内
                memory_used, gpu_util = gpu_info[i]
                print(f"检查GPU {i}: 显存使用 {memory_used}MB, 利用率 {gpu_util}%")
                # 判断条件：显存占用低于阈值且GPU利用率接近0
                if memory_used < memory_threshold and gpu_util < 5:
                    print(f"选择空闲GPU: cuda:{i}")
                    print(f"显存占用: {memory_used}MB, GPU利用率: {gpu_util}%")
                    return torch.device(f"cuda:{i}")
        
        print("没有检测到空闲GPU，使用CPU。")
        return torch.device("cpu")
        
    except Exception as e:
        print(f"检测GPU时出错：{e}，使用CPU。")
        return torch.device("cpu")

In [6]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
import torch

device = pick_free_gpu()

cache_dir = "/raid/gfc/llm/models"
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall", cache_dir=cache_dir)
model = GPT2LMHeadModel.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall", cache_dir=cache_dir).to(device)
text_generator = TextGenerationPipeline(model, tokenizer,device=device)   
text_generator("这是很久之前的事情了", max_length=100, do_sample=True)

当前GPU状态：
GPU 0: 显存使用 13421MB, 利用率 71%
GPU 1: 显存使用 20777MB, 利用率 59%
GPU 2: 显存使用 12861MB, 利用率 5%
GPU 3: 显存使用 3MB, 利用率 0%
GPU 4: 显存使用 20341MB, 利用率 0%
GPU 5: 显存使用 3MB, 利用率 0%
GPU 6: 显存使用 3MB, 利用率 0%
GPU 7: 显存使用 3MB, 利用率 0%
检查GPU 7: 显存使用 3MB, 利用率 0%
选择空闲GPU: cuda:7
显存占用: 3MB, GPU利用率: 0%


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '这是很久之前的事情了 ， 但 是 从 去 年 开 始 它 就 慢 慢 失 去 了 兴 趣 ， 我 开 始 慢 慢 发 现 自 己 在 那 方 面 不 足 ， 那 个 时 候 看 起 来 ， 我 感 觉 不 是 在 那 个 地 方 做 了 什 么 努 力 ， 不 努 力 才 是 最 重 要 的 。 在 我 的 坚 持 下 ， 一 些 工 作 也 慢 慢 变 的 更 加 努 力 了'}]

In [7]:
def collate_fn(batch):
    data = tokenizer.batch_encode_plus(batch, 
                                       padding=True, 
                                       truncation=True, 
                                       max_length=512, 
                                       return_tensors="pt")
    data['labels'] = data['input_ids'].clone()
    return data

loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=16, shuffle=True, drop_last=True, collate_fn=collate_fn)
print(len(loader))

13597


In [9]:
Epochs = 30

# 定义训练函数
def train():
    model.train()

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Epochs)
    # 梯度裁剪
    max_grad_norm = 1.0

    # 记录最佳模型
    best_loss = float('inf')

    for epoch in range(Epochs):
        total_loss = 0
        num_batches = 0
        for i, batch in enumerate(loader):
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            loss.backward()

            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
              
            optimizer.step()
            optimizer.zero_grad()
            
            if i % 100 == 0:
                with torch.no_grad():
                    # 计算准确率
                    model.eval()
                    labels = batch['labels'][:, 1:].to(device)
                    out = outputs.logits.argmax(dim=2)[:, :-1]

                    select = labels != 0
                    labels = labels[select]
                    out = out[select]
                    

                    acc = (labels == out).sum().item() / labels.numel()
                    lr = optimizer.param_groups[0]['lr']
                    print(f"Epoch {epoch}, Step {i}, Lr {lr:.5e}, Loss {loss:.5f}, Acc {acc:.2%}")

                    del select
            total_loss += loss.item()
            num_batches += 1

        # 每个epoch结束后的操作
        avg_loss = total_loss / num_batches
        print(f"Epoch {epoch} completed. Average Loss: {avg_loss:.5f}")

        # 调整学习率
        scheduler.step()

        # 保存模型
        if avg_loss < best_loss:
            best_loss = avg_loss
            model_save_path = "/raid/gfc/llm/params/gpt_project"
            os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': best_loss,
            }, os.path.join(model_save_path, "best_model.pt"))         
            

: 

In [ ]:
# train() # 全量微调gpt2！！！

### 对比原装pipeline 和 自定义pipeline的区别

### 1. 使用原始数据集进行训练的效果，发现数据集里面很多脏数据。。。

In [40]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline
import torch

cache_dir = "/raid/gfc/llm/models"

device = torch.device("cuda:4")

# 加载原装模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall", cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained("uer/gpt2-distil-chinese-cluecorpussmall", cache_dir=cache_dir).to(device)

finetuned_model_weights_path = "/raid/gfc/llm/params/gpt_project/best_model.pt"
# 加载保存的模型参数
checkpoint = torch.load(finetuned_model_weights_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"成功从{finetuned_model_weights_path}加载参数")

print("=" * 88)
print("模型结构如下")
print(model)
print("=" * 88)

original_pipeline = TextGenerationPipeline(model, tokenizer, device=device)

# 测试一些古诗文开头
test_prompts = [
    "天高",
    "床前明月光"
]

# 原装pipeline
print("=== 原装pipeline微调后模型生成结果 ===")
for prompt in test_prompts:
    print(f"\n输入: {prompt}")
    result = original_pipeline(prompt, max_length=100, do_sample=True, temperature=0.7)
    print(f"输出: {result[0]['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


成功从/raid/gfc/llm/params/gpt_project/best_model.pt加载参数
模型结构如下
GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=21128, bias=False)
)
=== 原装pipeline微调后模型生成结果 ===

输入: 天高
输出: 天高 风 急 ， 

In [42]:

# 自定义古诗词生成 Pipeline 生成五言绝句
def PoetryGenerationPipeline(text, row, col):
    # text提示词，row行数，col每一行字符数
    # 定义一个递归函数，用于生成文本
    def generate_text(data):
        with torch.no_grad():
            # print(data.input_ids.shape) # [batch_size, seq_len]
            out = model(**data) # [batch_size, seq_len, hidden_size]
            # for k in out.keys():
            #     print(k) # loss logits past_key_values
            # print(out.logits.shape) # torch.Size([2, 3, 21128])

            # 得到最后一个字符的预测概率，因为第二维原本是3，我要预测下一个字符，所以只取最后一个
            # out = out.logits
            # out = out[:, -1] # torch.Size([2, 21128])
            # 这两行代码和下面作用一样的，下面这个更清晰
            last_token_prob = out.logits[:, -1, :] # torch.Size([2, 21128])
            # print(out.shape)

            top_k_values = torch.topk(last_token_prob, 50).values
            # print(top_k_values.shape) # [batch_size, 50]
            # print(top_k_values)
            # 获取每个输出序列中前50个最大的logits（为保持维度不变，需要增加一个维度）
            top_k_values = top_k_values[:,-1].unsqueeze(dim=1)
            # print(top_k_values.shape) # [batch_size, 1]
            # print(top_k_values)
            
            # 屏蔽低概率词
            last_token_prob = last_token_prob.masked_fill(last_token_prob < top_k_values, -float('inf'))

            # 屏蔽特殊符号
            for sign in "，。、；‘【】、《》？：“{}|,./;'[]?{}`~@\\#￥%……&*（）——+!@#$%^&*()_+":
                if sign in tokenizer.get_vocab():
                    out.logits[:, :, tokenizer.get_vocab()[sign]] = -float('inf')
            # 屏蔽字母
            for letter in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ":
                if letter in tokenizer.get_vocab():
                    out.logits[:, :, tokenizer.get_vocab()[letter]] = -float('inf')
            # 屏蔽数字
            for number in "0123456789":
                if number in tokenizer.get_vocab():
                    out.logits[:, :, tokenizer.get_vocab()[number]] = -float('inf')

            # 采样，无放回，避免生成重复内容
            last_token_prob = last_token_prob.softmax(dim=1) # torch.Size([2, 21128]) 得到归一化概率

            last_token_prob = last_token_prob.softmax(dim=1)  # [2, 21128]
            c = data['input_ids'].shape[1] / (1 + col)
            if c % 1 == 0:
                last_token_prob.fill_(0)  # 将所有概率清零
                if c % 2 == 0:
                    last_token_prob[:, tokenizer.get_vocab()["。"]] = 1.0  # 强制生成 "。"
                else:
                    last_token_prob[:, tokenizer.get_vocab()["，"]] = 1.0  # 强制生成 "，"

            last_token_prob = last_token_prob.multinomial(num_samples=1) # torch.Size([2, 1]) 采样

            # # 添加标点符号
            # c = data['input_ids'].shape[1] / (1 + col)
            # if c % 1 == 0:
            #     if c % 2 == 0:
            #         # 表示将第一个位置（索引0）的概率设为1，我们强制模型在这个位置生成对应的标点符号
            #         last_token_prob[:,0] = tokenizer.get_vocab()["。"]
            #     else:
            #         last_token_prob[:,0] = tokenizer.get_vocab()["，"]

            # 更新 input_ids，将新产生的词添加到输入序列中
            data['input_ids'] = torch.cat([data['input_ids'], last_token_prob], dim=1)
            data['attention_mask'] = torch.ones_like(data['input_ids'])
            data['token_type_ids'] = torch.zeros_like(data['input_ids'])
            data['labels'] = data['input_ids'].clone()
            if data['input_ids'].shape[1] >= row * col + row + 1:
                return data
            return generate_text(data)
        
    # 测试
    # 1. 对输入文本进行编码
    # 得到[batch_size, seq_len]的一个矩阵，其中seq_len = [CLS] + token字符数
    data = tokenizer.batch_encode_plus(text, return_tensors="pt", padding=True, truncation=True)
    # 2. 移除编码后的序列中最后一个结束符号token
    data['input_ids'] = data['input_ids'][:, :-1].to(device)
    # 3. 创建与input_ids同形状的attention_mask
    data['attention_mask'] = torch.ones_like(data['input_ids']).to(device)
    # 4. 创建与input_ids同形状的token_type_ids
    data['token_type_ids'] = torch.zeros_like(data['input_ids']).to(device)
    # 5. 创建与input_ids同形状的labels
    data['labels'] = data['input_ids'].clone().to(device)
    data = generate_text(data)

    for i in range(len(text)):
        print(i, tokenizer.decode(data["input_ids"][i]))

PoetryGenerationPipeline(test_prompts, 4, 5)

0 [CLS] 天 高 [SEP] [PAD] [PAD] ， 匆 吩植 mp4 枢 。 safari 池 kevin 蹣jing ， 猥俨亨縝 引 。
1 [CLS] 床 前 明 月 光 ， 苦墉 オーフン5 粕橙 。什mark嘗 niusnews 混 ， sars遗チ才 谣 。


In [33]:
tokenizer.get_vocab()["。"]

511